In [ ]:
filePath = "..\\..\\Files\\"

In [ ]:
# Check that each file isn't the same size
import os

def list_files_by_size(folder_path):
    # List to store file names and their sizes
    files_info = []
    
    # Iterate through files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        # Check if it's a file
        if os.path.isfile(file_path):
            file_size = os.path.getsize(file_path)
            files_info.append((file_name, file_size))
    
    # Sort files by size (ascending order)
    sorted_files = sorted(files_info, key=lambda x: x[1])
    
    # Print sorted files with sizes
    for file_name, file_size in sorted_files:
        print(f"{file_name}: {file_size} bytes")

list_files_by_size(filePath)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from collections import Counter
from datetime import datetime
from pprint import pprint
import math

### Scan for pnr
#\b\d{12}\b|\d{8}-\w{4}

In [ ]:
reloadDataFiles = True

# Import files

In [ ]:
def LoadDrFile(initials):
    drDfVisists = pd.read_excel(f"{filePath}{initials}_besök.xlsx")
    drDfPhone = pd.read_excel(f"{filePath}{initials}_telefon.xlsx")
    drDfAdmin = pd.read_excel(f"{filePath}{initials}_indirekt.xlsx")
    drDfDistans = pd.read_excel(f"{filePath}{initials}_distans.xlsx")
    drDfDistans.loc[:, "Variabelnamn i M4"] = "Telefon"
    
    # Append all DataFrames into one
    drDf = drDfVisists.append(drDfPhone, ignore_index=True)
    drDf = drDf.append(drDfDistans, ignore_index=True)
    drDf = drDf.append(drDfAdmin, ignore_index=True)
    
    # Limit columns and filter out rows with null in 'Personnr'
    limitColsDf = drDf.loc[:, ["Personnr", "Variabelnamn i M4", "Besöksdatum"]]
    nonNaNDf = limitColsDf.loc[limitColsDf['Personnr'].notnull()].copy()  # Explicitly create a copy
    nonNaNDf["Personnr"] = nonNaNDf["Personnr"].apply(lambda x: f"{int(float(x)):012}" if isinstance(x, (float, int)) else f"{x:>012}")
    
    # Add the 'Initials' column
    nonNaNDf.loc[:, "Initials"] = initials
    
    return nonNaNDf


In [ ]:
if reloadDataFiles:
    JK = LoadDrFile("JK")
    MT = LoadDrFile("MT")
    LW = LoadDrFile("LW")
    RA = LoadDrFile("RA")
    MS = LoadDrFile("MS")
    MN = LoadDrFile("MN")
    DH = LoadDrFile("DH")
    AE = LoadDrFile("AE")
    IW = LoadDrFile("IW")
    LYY = LoadDrFile("LYY")

In [ ]:
# if reloadDataFiles:
#     print(RA.loc[800, :])

In [ ]:
if reloadDataFiles:
    listingOriginalDf = pd.read_csv(f"{filePath}Listing_2024-11-16.csv", sep=";")
    listingDf = listingOriginalDf.loc[:, ["Pnr", "Vårdgivare"]]
    listingDf.head(2)

In [ ]:
def read_tab_separated_file(file_path):
    result_dict = {}
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # Ignorera tomma rader
                if line.strip():
                    key, value = line.strip().split('\t')
                    result_dict[key] = value
    except FileNotFoundError:
        print(f"Filen {file_path} kunde inte hittas.")
    except ValueError:
        print("Fel format i filen. Kontrollera att varje rad innehåller exakt en tab.")

    return result_dict


In [ ]:
DrToInitials = read_tab_separated_file(filePath+"Permanent\\DrToInitials_ListingAnalysisOld.txt")

In [ ]:
if reloadDataFiles:
    listingDf.rename(columns={"Pnr": "Personnr", "Vårdgivare": "Listing"}, inplace=True)
    listingDf["Listing"] = listingDf["Listing"].apply(lambda name: DrToInitials[name] )
    listingDf.head()

In [ ]:
if reloadDataFiles:
    listingDict = dict(zip(listingDf["Personnr"], listingDf["Listing"]))

## Lägg ihop alla besök med listad läkar som egen kolumn

In [ ]:
if reloadDataFiles:
    allContacts = JK.append(MT, ignore_index=True)
    allContacts = allContacts.append(LW, ignore_index=True)
    allContacts = allContacts.append(RA, ignore_index=True)
    allContacts = allContacts.append(MS, ignore_index=True)
    allContacts = allContacts.append(MN, ignore_index=True)
    allContacts = allContacts.append(DH, ignore_index=True)
    allContacts = allContacts.append(AE, ignore_index=True)
    allContacts = allContacts.append(LYY, ignore_index=True)
    allContacts = allContacts.append(IW, ignore_index=True)

allContacts = allContacts.copy()

In [ ]:
if reloadDataFiles:
    allContacts["Personnr"] = allContacts["Personnr"].str.replace(r"(\d{8})(\w{4})", r"\1-\2")

In [ ]:
if reloadDataFiles:
    print(set(allContacts["Variabelnamn i M4"]))

In [ ]:
visitTypeDict = {
    'Indirekt kontakt': "adm",
    'Planerade mottagningsbesök': "visit",
    'Planerat hembesök': "visit",
    'Telefon': "tel"
}

In [ ]:
if reloadDataFiles:
    allContacts.rename(columns = {"Variabelnamn i M4" : "VisitType"}, inplace=True)
    allContacts["VisitType"] = allContacts["VisitType"].apply(lambda x: visitTypeDict[x])
#     print(allContacts.head(1))

In [ ]:
def checkListing(personnr):
    if not personnr in listingDict:
        return "unknown"
    
    return listingDict[personnr]

In [ ]:
if reloadDataFiles:
    allContacts["Listing"] = allContacts["Personnr"].apply( checkListing )

In [ ]:
if reloadDataFiles:
    allContacts.to_excel(f"{filePath}allContacts.xlsx", index=False)
else:
    allContacts = pd.read_excel(f"{filePath}allContacts.xlsx")
    allContacts.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], inplace=True)

In [ ]:
if reloadDataFiles:
    listingDf["Visits (n)"] = listingDf["Personnr"].apply(
        lambda personnr: len(allContacts.loc[ 
            (allContacts["Personnr"] == personnr) & 
            (allContacts["VisitType"] == "visit")].index))

    listingDf["Tel (n)"] = listingDf["Personnr"].apply(
        lambda personnr: len(allContacts.loc[ 
            (allContacts["Personnr"] == personnr) & 
            (allContacts["VisitType"] == "tel")].index))

    listingDf["Adm (n)"] = listingDf["Personnr"].apply(
        lambda personnr: len(allContacts.loc[ 
            (allContacts["Personnr"] == personnr) & 
            (allContacts["VisitType"] == "adm")].index))
    
    listingDf.to_excel(f"{filePath}listingDf.xlsx")
else:
    listingDf = pd.read_excel(f"{filePath}listingDf.xlsx")
    listingDf.drop(columns=["Unnamed: 0"], inplace=True)

### Lägg till antalet besök per listad pat i listingDf

In [ ]:
listingDf.loc[:, ["Listing", "Visits (n)", "Tel (n)", "Adm (n)"]].head(2)

In [ ]:
unlistedDf = listingDf.loc[ listingDf["Listing"] == "HC" ]
print(len(unlistedDf.index))


In [ ]:
#allContacts.loc[ allContacts["Personnr"] == "some_nr" ]

## Filtrera bort patienter som redan är listade

In [ ]:
unlistedContacts = allContacts.loc[ allContacts["Listing"] == "HC" ]

In [ ]:
unlistedContacts.to_excel(f"{filePath}unlistedContacts.xlsx")

In [ ]:
len(set(unlistedContacts.loc[:,"Personnr"]))

## Ta fram inaktiva patienter (listad på HC som ej haft besök sista åren)

In [ ]:
hasVisitSet = set( allContacts.loc[ allContacts["VisitType"] == "visit" ].loc[ : , "Personnr"] )

In [ ]:
print(f"There were {len(allContacts.index)} contacts in total.")
print(f"There were {len(allContacts.loc[ (allContacts['Listing'] == 'HC') ].index)} contacts from unlisted patients.")
unlistedHasContactSet = set( allContacts.loc[ (allContacts["Listing"] == "HC") ].loc[ : , "Personnr"] )
print(f"{len(unlistedHasContactSet)} unlisted patients had som form of contact.")
unlistedHasVisitSet = set( allContacts.loc[ (allContacts["Listing"] == "HC") & (allContacts["VisitType"] == "visit") ].loc[ : , "Personnr"] )
print(f"{len(unlistedHasVisitSet)} unlisted patients had a physical visit.")
unlistedHasTelSet = set( allContacts.loc[ (allContacts["Listing"] == "HC") & (allContacts["VisitType"] == "tel") ].loc[ : , "Personnr"] )
print(f"{len(unlistedHasTelSet)} unlisted patients had a tel appointment.")

In [ ]:
print(f"There are {len(listingDf.loc[ listingDf['Listing'] == 'HC' ].index)} unlisted patients in total")

In [ ]:
unlistedNoContactSet = listingDf.loc[ (listingDf["Listing"] == "HC") & ~(listingDf["Personnr"].isin(unlistedHasContactSet)) ]
#print(unlistedNoContactSet[:5])
print(len(unlistedNoContactSet))

In [ ]:
unlistedNoContactDf = unlistedDf.loc[ ~(unlistedDf["Personnr"].isin(unlistedHasContactSet)) ]
unlistedNoContactDf = unlistedNoContactDf.copy()
print(len(unlistedNoContactDf.index))
unlistedNoContactDf.loc[: , ["Listing", "Visits (n)", "Tel (n)", "Adm (n)"]].tail(3)

# Beräkningar

#### Använd:
* unlistedContacts - DF med alla olistade patienter som haft besök
* unlistedNoContactDf - DF med patienter som är olistade och inte har haft några kontakter
* unlistedContactsDf - DF med alla kontakter med olistade patienter
* unlistedHasContactSet - set med alla olistade patienter som har en kontakt

### Patient to visits per doctor df

In [ ]:
initialsList = ["MT", "JK", "LW", "QR", "MS", "MN", "RA"]
visitTypes = ["visit", "tel", "adm"]

In [ ]:
countAndScoreColNames = []
for initials in initialsList:
    for visitType in visitTypes:
        countAndScoreColNames.append(f"{initials}_{visitType}")
        countAndScoreColNames.append(f"{initials}_{visitType}_score")
    countAndScoreColNames.append(f"{initials}_score")
countAndScoreColNames.append(f"total_score")
countAndScoreColNames

In [ ]:
def countPatientsContactsWithDr(personnr, initials, visitType):
    return len(unlistedContacts.loc[ (unlistedContacts["Personnr"] == personnr) & 
                                   (unlistedContacts["Initials"] == initials) &
                                   (unlistedContacts["VisitType"] == visitType)].index)

#### Vikta besöken efter hur längesen de skedde.

In [ ]:
# Weight factor where a visit which happened just now is worth 2 points
# one which happened 1 year ago 1.5 points
# and a bit less than 2 years ago 1 point.

def weightFactorFromDate(dateString):
    date = datetime.strptime(dateString, '%Y-%m-%d')
    originDate = datetime.strptime("2022-04-30", '%Y-%m-%d')
    dateDiff = (originDate - date).days
    return 2 - (dateDiff/650)
    
print(weightFactorFromDate("2022-04-30"))
print(weightFactorFromDate("2021-10-30"))
print(weightFactorFromDate("2021-04-30"))
print(weightFactorFromDate("2020-10-30"))
print(weightFactorFromDate("2020-07-30"))
print(weightFactorFromDate("2020-07-7"))

visitTypeWeightDict = {
    "visit": 1,
    "tel": 0.35,
    "adm": 0.2,
}

In [ ]:
def calculateRowScore(row):
    dateWeightFactor = weightFactorFromDate(row["Besöksdatum"])
    return visitTypeWeightDict[ row["VisitType"] ] * dateWeightFactor

testRow = unlistedContacts.loc[ list(unlistedContacts.index)[1200] , :]
pprint(testRow.loc[list(set(testRow.index) - set(["Personnr"]))])
print(f"Score: {calculateRowScore( testRow )}")

In [ ]:
unlistedContacts["rowScore"] = unlistedContacts.apply(calculateRowScore, axis=1)

In [ ]:
unlistedContacts.loc[:, ["VisitType", "Besöksdatum", "Initials", "Listing", "rowScore"]].head(2)

In [ ]:
unlistedContacts.columns

In [ ]:
def createPatientRowOfVisitsPerDr(personnr):
    patientsIndividualDf = unlistedContacts.loc[ unlistedContacts["Personnr"] == personnr ]
    
    row = {"Personnr": personnr}
    totalScore = 0
    
    for initials in initialsList:
        
        drIndividualDf = patientsIndividualDf.loc[ patientsIndividualDf["Initials"] == initials ]
        
        individualScore = 0
        
        for visitType in visitTypes:
            
            visitTypeDf = drIndividualDf.loc[ drIndividualDf["VisitType"] == visitType ]
            row[f"{initials}_{visitType}"] = len( visitTypeDf.index )
            score = visitTypeDf["rowScore"].sum()
            row[f"{initials}_{visitType}_score"] = score
            individualScore += score
            
        row[f"{initials}_score"] = individualScore
        totalScore += individualScore
    row[f"total_score"] = totalScore
    return row

In [ ]:
patientVisitPerDrList = []
limit = -1
i = 0
for personnr in unlistedHasContactSet:
    i += 1
    if limit > 0 and i > limit:
        break
    
    patientVisitPerDrList.append(createPatientRowOfVisitsPerDr(personnr))
    
    
unlistedPatientScoreDf = pd.DataFrame(patientVisitPerDrList, columns=["Personnr"] + countAndScoreColNames)
unlistedPatientScoreDf.loc[:, countAndScoreColNames].head()

In [ ]:
def countDrAmountOfPatientsForEachVisitAmount(initials):
    counter = Counter(unlistedPatientScoreDf[f"{initials}_visit"])
    return dict(counter)
    #return sorted(counter.items())

In [ ]:
unlistedVisistsPerDrDict = {}
for initials in initialsList:
    unlistedVisistsPerDrDict[initials] = countDrAmountOfPatientsForEachVisitAmount(initials)
    
unlistedVisistsPerDrDict

In [ ]:
unlistedVisistsPerDrList = []
for i in range(6):
    #row = {"Visits (n)": i}
    row = {}
    for initials in initialsList:
        if i in unlistedVisistsPerDrDict[initials]:
            row[initials] = unlistedVisistsPerDrDict[initials][i]
        else:
            row[initials] = 0
    unlistedVisistsPerDrList.append(row)
    
unlistedVisistsPerDrDf = pd.DataFrame(unlistedVisistsPerDrList, columns=initialsList)
unlistedVisistsPerDrDf

### Extremt aktiva till JK och MT

In [ ]:
print("Histogram of total score")
unlistedPatientScoreDf.loc[:, "total_score"].hist()

### Most active patients

In [ ]:
def ageFromPersonnr(personnrStr):
    match = re.search(r"(\d{4})", personnrStr)
    year = int(match[1])
    return 2022 - year
    
ageFromPersonnr("19870323-abcd")

In [ ]:
ageAndScoreDf = unlistedPatientScoreDf.loc[:, ["Personnr", "total_score"]].copy()

In [ ]:
ageAndScoreDf["age"] = ageAndScoreDf["Personnr"].apply(ageFromPersonnr)

## Age distribution of patients

In [ ]:
ageAndScoreDf["age"].hist()

### Contact score per age

In [ ]:
test = np.array([1,0,0,0.25])
test.reshape(1, -1)
ageAndScoreDf.plot.scatter(x="age", y="total_score", c=test )

In [ ]:
scoreLimitForHeavy = 5
unlistedHeavySet = set( unlistedPatientScoreDf.loc[ unlistedPatientScoreDf["total_score"] >= scoreLimitForHeavy ].loc[ : ,"Personnr"] )
heavyN = len(unlistedHeavySet)
unlistedN = len(unlistedDf.index)
print(f"Heavy ({heavyN}) / unlisted {unlistedN} = {100 * heavyN / unlistedN} %")

#### Extremely active to JK & MT

In [ ]:
def GetExtremelyActiveForDr(initials, scoreLimit, otherAllowedFraction):
    resultDf = unlistedPatientScoreDf.loc[ (unlistedPatientScoreDf[f"{initials}_score"] > scoreLimit) |
                                           (unlistedPatientScoreDf[f"{initials}_visit"] >= 3) ]

    otherInitialsList = ["RA", "QR", "MS", "MN", "LW"]

    for otherInitials in otherInitialsList:
        resultDf = resultDf.loc[ resultDf[f"{otherInitials}_score"] < resultDf[f"{initials}_score"] * otherAllowedFraction ]

    return resultDf.loc[:, [
                 'Personnr',
                 f'{initials}_visit',
                 f'{initials}_visit_score',
                 f'{initials}_tel',
                 f'{initials}_tel_score',
                 f'{initials}_adm',
                 f'{initials}_adm_score',
                 f'{initials}_score',
                 'total_score',]]

In [ ]:
extremelyActiveJK = GetExtremelyActiveForDr("JK", 3, 0.4)

In [ ]:
extremelyActiveMT = GetExtremelyActiveForDr("MT", 3, 0.4)

### Patients with contacts but no visits?

In [ ]:
unlistedContactNoVisitDf = unlistedDf.loc[ (unlistedDf["Visits (n)"] == 0) & ((unlistedDf["Tel (n)"] > 0) | (unlistedDf["Adm (n)"] > 0)) ]
print(len(unlistedContactNoVisitDf))
unlistedContactNoVisitDf.loc[:, ["Listing", "Visits (n)", "Tel (n)", "Adm (n)"]].head()

## Calculate target amount of listings per dr except for MT and JK

#### First, calculate current amount of listed patients and how many of those are actively seeking care

In [ ]:
listingDf.loc[:, ["Listing", "Visits (n)", "Tel (n)", "Adm (n)"]].head(2)

In [ ]:
def getAmountsListedPerDr(minVisits):
    amountListedPerDr = {}
    for initials in initialsList:
        amount = len(listingDf.loc[ (listingDf["Listing"] == initials) &
                                            (listingDf["Visits (n)"] >= minVisits) ].index)
        amountListedPerDr[initials] = amount
        
    return amountListedPerDr
    

In [ ]:
print(getAmountsListedPerDr(0))

### For each patient, determine which dr it should go to:
* Never JK or MT
* The one with the highets score.
* Adjusted by a factor for each dr to try to reach the target amount

In [ ]:
print(f"Unlisted with som contact {len(unlistedHasContactSet)} and with a visit {len(unlistedHasVisitSet)}");

## How many do we have space to list and how many do we need to enlist?

In [ ]:
onePlusVisitPatienAmount = len(unlistedDf.loc[ unlistedDf["Visits (n)"] >= 1 ].index)
twoPlusVisitPatienAmount = len(unlistedDf.loc[ unlistedDf["Visits (n)"] >= 2 ].index)
print(f"Totalt antal olistade patienter som sökt 1+ gånger sedan 2021-07: {onePlusVisitPatienAmount}")
print(f"Totalt antal olistade patienter som sökt 2+ gånger sedan 2021-07: {twoPlusVisitPatienAmount}")

In [ ]:
def showListingInfo(initialsToShowList, expectedAmountListedPerDr, total):
    for initials in initialsToShowList:
        value = expectedAmountListedPerDr - tempDict[initials]
        print (f"{initials} har listat {tempDict[initials]}/{expectedAmountListedPerDr}. Återstår att lista: {value}")
        data = {}
        data["currentlyListed"] = tempDict[initials]
        data["expected"] = expectedAmountListedPerDr
        data["slotsLeft"] = value
        openSlotsPerDr[initials] = data
        total["counter"] += value
        total["places"] += expectedAmountListedPerDr

In [ ]:
openSlotsPerDr = {}
tempDict = getAmountsListedPerDr(0)
total = {"counter": 0, "places": 0}

showListingInfo(["MS", "MN", "RA"], 500, total)
showListingInfo(["MT", "JK", "LW", "QR"], 1000, total)
    
print(f"Total finns plats att lista {total['counter']} patienter")

In [ ]:
freeSpaces = total['counter']
expectedTotalListings = total["places"]
totalUnlisted = len(unlistedDf.index)
totalListed = len(listingDf.loc[ listingDf["Listing"] != "HC" ].index)
totalPat = len(listingDf.index)

print(f"""
Totalt antal patienter listade på enheten: {totalPat}
Totalt antal läkar-listade patienter: {totalListed}
Totalt antal läkar-olistade patienter: {totalUnlisted}
Andelen listade: {100*totalListed/totalPat:.0f} %
Antal som behöver listas 75 %: {0.75*totalPat - totalListed:.0f}
Antal som behöver listas 80 %: {0.8*totalPat - totalListed:.0f}
Procent om vi listar {freeSpaces}: {100*(expectedTotalListings)/totalPat:.3f} %""")


### Room for active patients:

In [ ]:
openSlotsPerDr

In [ ]:
activePatientsToBeListedN = len(unlistedPatientScoreDf.index)
roomForActive = 0

for initials in set(initialsList) - set(["MT", "JK"]):
    slotsLeft = openSlotsPerDr[initials]["slotsLeft"]
    roomForActive += slotsLeft
    
print(f"Room for active patient enlisting: {roomForActive}")
print(f"Amount of active patients: {activePatientsToBeListedN}")
print(f"Amount of nonactive patients to be divided between other than MT and JK:{roomForActive - activePatientsToBeListedN}")
print(f"Per 1000 pat dr:{2 * (roomForActive - activePatientsToBeListedN) / 7:.0f} and per 500 pat dr: {(roomForActive - activePatientsToBeListedN) / 7:.0f}")

In [ ]:
stInactiveN = 22
specialistInactiveN = 43

In [ ]:
unlistedPatientScoreDf.sort_values(by="total_score", ascending=False, inplace=True)

# Move patients from unlisted to enlisting

In [ ]:
def printCurrentStats():
    print(  f"""Unlisted left: {len(totalUnlistedLeftSet)}
Active unlisted left: {len(activeUnlistedLeftDf.index)}
Inactive unlisted left: {len(inactiveUnlistedLeftSet)}
Enlisting: {len(enlistingDf.index)}""")
    for initials in initialsList:
        print(f"Enlisting ({initials}): {len(enlistingDf.loc[ enlistingDf['TargetDr']  == initials ])}")

In [ ]:
def removePatientSetFromRemaining(removalSet):
    global activeUnlistedLeftDf, totalUnlistedLeftSet, inactiveUnlistedLeftSet
    totalUnlistedLeftSet -= removalSet
    if removalSet.issubset(inactiveUnlistedLeftSet):
        inactiveUnlistedLeftSet -= removalSet
    if removalSet.issubset(set(activeUnlistedLeftDf.index)):
        activeUnlistedLeftDf = activeUnlistedLeftDf.loc[ set(activeUnlistedLeftDf.index) - removalSet, : ]

In [ ]:
# def removePatientFromRemaining(personnrStr):
#     totalUnlistedLeftSet.remove(personnrStr)
#     if personnrStr in inactiveUnlistedLeftSet:
#         inactiveUnlistedLeftSet.remove(personnrStr)
#     if personnrStr in activeUnlistedLeftDf.index:
#         activeUnlistedLeftDf.drop(personnrStr, inplace=True)

In [ ]:
def performEnlisting(initials, enlistingSet):
    global enlistingDf
    for personnr in enlistingSet:
        row = scoreDf.loc[personnr, :]
        row["TargetDr"] = initials
        enlistingDf = enlistingDf.append(row)
    removePatientSetFromRemaining(enlistingSet)

## Start with active patients
<img src="https://c8.alamy.com/comp/2FWKCW4/math-formulas-mathematical-formulas-on-green-school-chalkboard-handwritten-scientific-math-equations-theories-or-calculations-vector-background-algebra-and-geometry-education-concept-2FWKCW4.jpg" style="max-height:300px">

In [ ]:
scoreDf = unlistedPatientScoreDf.set_index("Personnr")
totalUnlistedLeftSet = set(unlistedDf["Personnr"])
inactiveUnlistedLeftSet = set(unlistedNoContactDf["Personnr"])
activeUnlistedLeftDf = scoreDf.copy().sample(frac=1)
enlistingDf = pd.DataFrame(columns = list(unlistedPatientScoreDf.columns) + ["TargetDr"])
printCurrentStats()

# disable chained assignments
pd.options.mode.chained_assignment = None 

### Move patients designated for JK and MT first

In [ ]:
performEnlisting("JK", set(extremelyActiveJK["Personnr"]))
printCurrentStats()

In [ ]:
performEnlisting("MT", set(extremelyActiveMT["Personnr"]))
printCurrentStats()

## Then active patients to others

In [ ]:
for initials in set(initialsList) - {"MT", "JK"}:
    entry = openSlotsPerDr[initials]
    inactiveN = stInactiveN if entry["expected"] == 500 else specialistInactiveN
    if initials == "MS":
        inactiveN = 0
    entry["targetNewActive"] = entry["slotsLeft"] - inactiveN
    
openSlotsPerDr

In [ ]:
orderToEnlistDrs = pd.DataFrame(columns=["Initials", "TargetNewActive"])

In [ ]:
orderToEnlistDrsList = []
for key, entry in openSlotsPerDr.items():
    if key not in ["MT", "JK"]:
        row = {"Initials": key, "TargetNewActive": entry["targetNewActive"]}
        orderToEnlistDrsList.append(row)
    
orderToEnlistDrsDf = pd.DataFrame(orderToEnlistDrsList)
orderToEnlistDrsDf.sort_values("TargetNewActive", inplace=True)
orderToEnlistDrsList = orderToEnlistDrsDf.values.tolist()
orderToEnlistDrsList = [entry[0] for entry in orderToEnlistDrsList]
orderToEnlistDrsList

In [ ]:
def evaluatePatientForDr(personnr, row, initials, factor, priorityOthers, debug):
    maxOtherScore = 0
    maxOtherInitials = "<none>"
    priorityOthers = set(priorityOthers)
    
    for otherInitials in (set(initialsList) - set({initials})):
        otherScore = row[f"{otherInitials}_score"]
        ownScore = row[f"{initials}_score"]
        marginFactor = 0.01
        ownScoreFactored = ownScore + factor
        priorityStr = ""
        priorityStr2 = ""
        
        if otherScore > 0.05 and otherInitials in priorityOthers:
            marginFactor = -0.01
            ownScoreFactored = ownScore + factor / 5
            priorityStr = "PRIORITY: "
            priorityStr2 = "(DESPITE OF PRIORITY) "
        
        if otherScore >= ownScoreFactored + marginFactor:
            if debug:
                print(f"{priorityStr}{otherInitials}_score ({otherScore}) > {initials}_score ({ownScore}) + {factor} ({ownScoreFactored})")
            return False
        
        if otherScore > maxOtherScore:
            maxOtherInitials = otherInitials
            maxOtherScore = otherScore
        
    if debug:
        print (f"Max other score was {priorityStr2}{maxOtherInitials} ({maxOtherScore}) < {initials}_score ({ownScore}) + {factor} ({ownScoreFactored})")
    return True

In [ ]:
def calculateEnlistingForDr(initials, amount, priorityOthers, simplifyFactor, debug):
    targetNewActive = amount # openSlotsPerDr[initials]["targetNewActive"]
    activeUnlistedLeftDf.sort_values(by=f"{initials}_score", ascending=False, inplace=True)

    enlistedCounter = 0
    safety = 0
    safetyLimit = 10
    factor = 0
    enlistingSet = set()
    while enlistedCounter < targetNewActive and safety < safetyLimit:
        if debug:
            print("\n\n##############################################")
            print("################ NEW ROUND ###################")
            print("##############################################\n")

        for tpl in activeUnlistedLeftDf.iterrows():
            personnr = tpl[0]
            row = tpl[1]
            doEnlist = evaluatePatientForDr(personnr, row, initials, factor + simplifyFactor, priorityOthers, debug)

            if doEnlist and personnr in enlistingSet:
                if debug:
                    print("Already enlisted")
            elif doEnlist:
                enlistedCounter += 1
                enlistingSet.add(personnr)

            if enlistedCounter >= targetNewActive:
                break

        safety += 1
        factor += 0.2

    if debug:
        print(f"\Selected {len(enlistingSet)} / {targetNewActive} patients for enlisting")
    
    return enlistingSet

In [ ]:
class Enlister:
    def __init__(self, initials):
        self.initials = initials
        self.targetActive = openSlotsPerDr[initials]["targetNewActive"]
        self.enlistedSoFar = 0
        self.isDone = False
        self.debug = False
        
    def enlistN(self, n, simplifyFactor):
        amount = n
        if self.enlistedSoFar + n > self.targetActive:
            amount = self.targetActive - self.enlistedSoFar
        
        enlistingSet = calculateEnlistingForDr(self.initials, amount, priorityOthers, simplifyFactor, self.debug)
        
        self.enlistedSoFar += len(enlistingSet)
        if (self.enlistedSoFar >= self.targetActive):
            self.isDone = True
        
        performEnlisting(self.initials, enlistingSet)
        removePatientSetFromRemaining(enlistingSet)
        
        return f"{self.initials}: Enlisted {len(enlistingSet)} => {self.enlistedSoFar} / {self.targetActive}: Done - {self.isDone}\n"
        
    def __str__(self):
        return f"Enlister({self.initials})"
    
    def __repr__(self):
        return f"Enlister({self.initials})"

In [ ]:
enlisters = []
for initials in orderToEnlistDrsList:
    enlister = Enlister(initials)
    enlisters.append(enlister)
    
print(enlisters)

In [ ]:
def checkIfAllDone():
    for enlister in enlisters:
        if not enlister.isDone:
            return False
    return True

In [ ]:
def getPriorityOthers():
    priorityOthers = set()
    for enlister in enlisters:
        if not enlister.isDone:
            priorityOthers.add(enlister.initials)
            
    return priorityOthers

In [ ]:
safety = 0
safetyLimit = 99
remainingLastTime = len(activeUnlistedLeftDf.index)
simplifyFactor = 0

while len(activeUnlistedLeftDf.index) > 30 and not checkIfAllDone() and safety <= safetyLimit:
    safety += 1
    remainingNow = len(activeUnlistedLeftDf.index)
    if (remainingNow == remainingLastTime):
        simplifyFactor += 0.2
    
    remainingLastTime = remainingNow
    
    outStr = f"\nACTIVE LEFT: {len(activeUnlistedLeftDf.index)}, ENLISTED: {len(enlistingDf.index)}\n"
    outStr += f"{getPriorityOthers()}\n"
    
    for enlister in enlisters:
        if not enlister.isDone:
            priorityOthers = getPriorityOthers()
            outStr += enlister.enlistN(20, simplifyFactor)
        
    print(outStr)
        

In [ ]:
toLwDf = activeUnlistedLeftDf.copy()
toLwDf.sort_values(by="LW_score", ascending=False, inplace=True)
enlistingSet = set(toLwDf.index)
toLwDf["TargetDr"] = "LW"
print(len(enlistingSet))
#toLwDf.loc[ list(toLwDf.index)[:2], ["LW_score", "total_score"] ]

In [ ]:
enlistingDf = enlistingDf.append(toLwDf, sort=False)
removePatientSetFromRemaining(enlistingSet)
printCurrentStats()

## Compare how heavy burden each one gets

In [ ]:
for initials in initialsList:
    mean = enlistingDf.loc[ enlistingDf["TargetDr"] == initials ]["total_score"].mean()
    print(f"{initials}: {mean}")

# Add the inactive ones at random

In [ ]:
rowTemplate = dict(zip(countAndScoreColNames, [0 for x in countAndScoreColNames]))
rowTemplate["TargetDr"] = ""
rowTemplate["Personnr"] = ""

In [ ]:
unlistedNoContactDf = unlistedNoContactDf.sample(frac=1) # Shuffle
unlistedNoContactList = list(unlistedNoContactDf["Personnr"])

In [ ]:
for initials in initialsList:
    entry = openSlotsPerDr[initials]
    activeEnlisting = len(enlistingDf.loc[ enlistingDf["TargetDr"] == initials ].index)
    entry["currentlyEnlisting"] = activeEnlisting
    entry["targetInactive"]  = entry["slotsLeft"] - activeEnlisting
    print(f"targetInactive {initials}: {entry['targetInactive']}")


In [ ]:
def enlistOneInactivePat(initials):
    personnr = unlistedNoContactList.pop()
    row = dict(rowTemplate)
    row["TargetDr"] = initials
    series = pd.Series(row)
    enlistingDf.loc[personnr, :] = series
    return personnr

In [ ]:
for initials in initialsList:
    entry = openSlotsPerDr[initials]
    removeSet = set()
    targetInactive = entry["targetInactive"]
    
    for i in range(targetInactive):
        personnr = enlistOneInactivePat(initials)
        removeSet.add(personnr)
        
    removePatientSetFromRemaining(removeSet)
    print(f"{initials}: Enlisted {len(removeSet)}")
                          
#enlistingDf.tail(2)  

In [ ]:
printCurrentStats()

In [ ]:
for initials in initialsList:
    entry = openSlotsPerDr[initials]
    print(f"{initials}: {entry['slotsLeft']}")

## Sorting before export

In [ ]:
enlistingDf.sort_values(by="total_score", inplace=True, ascending=False)
enlistingDf.sort_values(by="TargetDr", inplace=True)

In [ ]:
outputDf = pd.DataFrame(columns=list(enlistingDf.columns))
for initials in initialsList:
    outputDf = outputDf.append(enlistingDf.loc[ enlistingDf["TargetDr"] == initials ].sort_values(by=f"{initials}_score", ascending=False))

# Exporting

In [ ]:
exportColumns = []
for initials in initialsList:
    exportColumns.append(f"{initials}_score")

exportColumns.append("total_score")
exportColumns.append("TargetDr")
exportColumns

In [ ]:
# outputDf.to_excel(f"{filePath}enlistingDf_allColumns.xlsx")
# outputDf.loc[:, exportColumns].to_excel(f"{filePath}enlistingDf_simplified.xlsx")

# Kuriosa

In [ ]:
allContactsCalc = allContacts.copy()
allContactsCalc["rowScore"] = allContactsCalc.apply(calculateRowScore, axis=1)

In [ ]:
currentListingDict = dict(zip(list(listingDf["Personnr"]), list(listingDf["Listing"])))

In [ ]:
futureListingDict = dict(zip(list(enlistingDf.index), list(enlistingDf["TargetDr"])))

In [ ]:
def getTargetListing(personnr):
    if personnr in currentListingDict and currentListingDict[personnr] != "HC":
        return currentListingDict[personnr]
    elif personnr in futureListingDict:
        return futureListingDict[personnr]
    else:
        return "<unknown>"

In [ ]:
allContactsCalc["TargetListing"] = allContactsCalc["Personnr"].apply(getTargetListing)

In [ ]:
Counter(allContactsCalc["TargetListing"])

In [ ]:
listingDf.columns

In [ ]:
def getTotalAmountOfListed(initials):
    fromEnlisting = set(enlistingDf.loc[ enlistingDf["TargetDr"] == initials ].index)
    fromListing = set( listingDf.loc[ listingDf["Listing"] == initials ]["Personnr"] )
    print(f"fromContacts: {len(fromEnlisting)}, eg. {list(fromEnlisting)[:3]}")
    print(f"fromListing: {len(fromListing)}, eg. {list(fromListing)[:3]}")
    return len(fromEnlisting.union(fromListing))
    
getTotalAmountOfListed("JK")

In [ ]:
calc = []
for initials in initialsList:
    
    initials = initials
    listedPats = getTotalAmountOfListed(initials)
    listedContactsAnyDr = len(allContactsCalc.loc[ (allContactsCalc["Listing"] == initials) ].index)
    listedVisitsAnyDr = len(allContactsCalc.loc[ (allContactsCalc["Listing"] == initials) & (allContactsCalc["VisitType"] == "visit") ].index)
    listedScoreAnyDr = allContactsCalc.loc[ (allContactsCalc["Listing"] == initials) ]["rowScore"].sum()
    actualVisits = len(allContactsCalc.loc[ (allContactsCalc["Initials"] == initials) & (allContactsCalc["VisitType"] == "visit") ].index)
    actualContacts = len(allContactsCalc.loc[ (allContactsCalc["Initials"] == initials) ].index)
    actualContactScore = allContactsCalc.loc[ (allContactsCalc["Initials"] == initials) ]["rowScore"].sum()
    scorePerListedVisit = listedScoreAnyDr / listedVisitsAnyDr
    actualVisitScorePerListedPat = actualContactScore / listedPats
    scorePerActualVisit = actualContactScore / actualVisits
    
    row = {
        "initials": initials,
        "listedPats": listedPats,
        "listedContactsAnyDr": listedContactsAnyDr,
        "listedVisitsAnyDr": listedVisitsAnyDr,
        "listedScoreAnyDr": listedScoreAnyDr,
        "actualVisits": actualVisits,
        "actualContacts": actualContacts,
        "actualContactScore": actualContactScore,
        "scorePerListedVisit": scorePerListedVisit,
        "actualVisitScorePerListedPat": actualVisitScorePerListedPat,
        "scorePerActualVisit": scorePerActualVisit,
    }
    calc.append(row)
    
calculationsDf = pd.DataFrame(calc, columns = ["initials", "listedPats", "listedContactsAnyDr", "listedVisitsAnyDr", "listedScoreAnyDr", "actualVisits", "actualContacts", "actualContactScore", "scorePerListedVisit", "actualVisitScorePerListedPat", "scorePerActualVisit"])
calculationsDf

In [ ]:
calculationsDf.to_excel(f"{filePath}calculations.xlsx")